#**Actividad 5 **

Curso MNA- NLP

Dr. Julio César Galindo

Vectores embebidos FasText para clasificación (Sentiment analysis)

In [ ]:
import pandas as pd
import numpy as np

import nltk
from nltk.corpus import stopwords

import re
import string

In [ ]:
nltk.download('punkt')    # es un tokenizador que ayuda a dividr el texto en enunciados mediante un modelo no-supervisado.
nltk.download('stopwords')    # para tener acceso a "stopwords" en varios idiomas.

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# Veamos la lista de stopwords que se incluyen de manera predeterminada la suite de librerías de NLTK:

mystopwords = stopwords.words('english')
print(len(mystopwords))
print(mystopwords)

179
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than

##**Pregunta - 1: cargamos los datos...**



Descarga los 3 archivos de Canvas. En particular, el archivo de datos de IMDb ya no requiere
transformarse para obtener sus 1000 registros. Al cargar los datos de los tres archivos deberás
tener un DataFrame de Pandas de 3000 registros, con sus etiquetas. Los archivos los encuentras en
Canvas y se llaman: amazon5.txt, imdb5.txt, yelp5.txt.


In [ ]:
# Puedes modificar en dado caso la ruta de los archivos:

dfa = pd.read_csv('amazon5.txt', sep='\t', names=['review','label'], header=None, encoding='utf-8')
dfi = pd.read_csv('imdb5.txt', delimiter='\t', names=['review','label'], header=None, encoding='utf-8')
dfy = pd.read_csv('yelp5.txt', sep='\t', names=['review','label'], header=None, encoding='utf-8')


print('Total de registros de Amazon:',dfa.shape)
print('Total de registros de IMBD:',dfi.shape)
print('Total de registros de Yelp:',dfy.shape)

Total de registros de Amazon: (1000, 2)
Total de registros de IMBD: (1000, 2)
Total de registros de Yelp: (1000, 2)


In [ ]:
dfa.head()     # Cada uno de estos data frame está formado de dos columnas,
               # uno con los comentarios (review) y otro con la etiqueta (label).
               # En particular aquí mostramos en el Amazon:

,review,label
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [ ]:
dfi.head()     # por alguna razón los labels de este archivo está separado por 4 tabs... habría que ver con algún
               # argumento de pd.read_csv() el poder cargarlo directamente... mientras tanto usemos regex par ajustarlo...

,review,label
0,"A very, very, very slow-moving, aimless movie ...",NaN
1,Not sure who was more lost - the flat characte...,NaN
2,Attempting artiness with black & white and cle...,NaN
3,Very little music or anything to speak of. 0,NaN
4,The best scene in the movie was when Gerardo i...,NaN


In [ ]:
dfii = pd.DataFrame({"review":["review"]*dfa.shape[0], "label":["label"]*dfa.shape[0]})

for k in range(dfii.shape[0]):
  dfii.iloc[k,:][0] = dfi.iloc[k,:][0][0:-5]   # tiene 4 tabs de más, aunque igualmente se pueden dejar ahí y luego limpiarlo.
  dfii.iloc[k,:][1] = float(dfi.iloc[k,:][0][-1])

dfii.head()

,review,label
0,"A very, very, very slow-moving, aimless movie ...",0.0
1,Not sure who was more lost - the flat characte...,0.0
2,Attempting artiness with black & white and cle...,0.0
3,Very little music or anything to speak of.,0.0
4,The best scene in the movie was when Gerardo i...,1.0


In [ ]:
dfii['label'] = pd.to_numeric(dfii['label'])   # para quitar algunos strings que hay por ahí en los labels...

In [ ]:
dfii.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   review  1000 non-null   object 
 1   label   1000 non-null   float64
dtypes: float64(1), object(1)
memory usage: 15.8+ KB


In [ ]:
dfy.head()

,review,label
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [ ]:
# y concatenamos los tres conjuntos para tener finalmente los 3000 registros:

df = pd.concat([dfa, dfii, dfy], ignore_index=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   review  3000 non-null   object 
 1   label   3000 non-null   float64
dtypes: float64(1), object(1)
memory usage: 47.0+ KB


##**Pregunta - 2:**

Realiza de nuevo un proceso de limpieza. Aplica el preprocesamiento que consideres adecuado, sin embargo, deberás aplicar necesariamente alguna de las técnicas de lematización.

Como aplicaremos modelos embebidos pre-entrenados, queremos palabras lo más cercanas a las existentes en un idioma, inglés en este caso.

Aplica y justifica cualquier otro proceso de limpieza que consideres
adecuado.

Recuerda que en esta actividad se usarán vectores embebidos para un problema de clasificación, por lo que deberás tomar de acuerdo a este contexto.

Justifica todas las transformaciones que se apliquen.









In [ ]:
# Separamos la información:
 #     La "X" serán los datos de entrada, los comentarios.
 #     La "Y" será la variable de salida, la evaluación.
 # Ambos, X y Y son "Series"

X = df.review     # Serie de strings
Y = df.label      # Serie de enteros 0s y 1s

assert X.shape == (3000,)           # verificando que tenemos la dimensiones esperadas.
assert Y.shape == (3000,)

In [ ]:
def clean_tok(doc):

  doc = re.sub('[!,\.:-]', ' ', doc)       # Primero separamos por aquellos casos que se omitieron espacios en blanco erróneamente.

  # tokenizamos por espacios en blanco cada reseña
  tokens = doc.split()

  # omitimos todos los signos de puntuación
  re_punc = re.compile('[%s]' % re.escape(string.punctuation))
  tokens = [re_punc.sub('', w) for w in tokens]

  # nos quedamos solo con letras y las transformamos a minúsculas
  tokens = [word.lower() for word in tokens if word.isalpha()]

  # omitamos las stopwords
  tokens = [w for w in tokens if not w in mystopwords]

  # apliquemos esto hasta después de la lemmatization:
  #tokens = [word for word in tokens if len(word) > 1]


  return tokens

In [ ]:
# Aplicamos el proceso de limpieza y tokenización:

Xcleantok = [clean_tok(x) for x in X]

In [ ]:
for x in Xcleantok[0:5]:
  print(x)

['way', 'plug', 'us', 'unless', 'go', 'converter']
['good', 'case', 'excellent', 'value']
['great', 'jawbone']
['tied', 'charger', 'conversations', 'lasting', 'minutes', 'major', 'problems']
['mic', 'great']


In [ ]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
import re

In [ ]:
# apliquemos la lematización que nos piden, puede variar con cada equipo:


def clean_lemma_doc(doc):

  wnl = WordNetLemmatizer()

  tokens = [wnl.lemmatize(w, pos='v') for w in doc]   # verbs
  tokens = [wnl.lemmatize(w, 'n') for w in tokens]    # nouns
  tokens = [wnl.lemmatize(w, 'a') for w in tokens]    # adverbs

  # omitimos caracteres de longitud menor a una determinada
  tokens = [word for word in tokens if len(word) > 1]


  return tokens

In [ ]:
# Aplicamos el proceso de limpieza/normalización adicionales:

Xclean = [clean_lemma_doc(x) for x in Xcleantok]

In [ ]:
for x in Xclean[0:5]:
  print(x)

['way', 'plug', 'unless', 'go', 'converter']
['good', 'case', 'excellent', 'value']
['great', 'jawbone']
['tie', 'charger', 'conversation', 'last', 'minute', 'major', 'problem']
['mic', 'great']


#**Pregunta 3:**

Llamar Xclean a los comentarios procesados y Y a las etiquetas.

Realicemos una partición aleatoria con los mismos porcentajes de la práctica pasada para poder comparar dichos resultados con los de
esta actividad, a saber, 70%, 15% y 15%, para entrenamiento, validación y prueba, respectivamente.

Verifica que obtienes 2100 registros de entrenamiento y 450 para cada uno de validación y prueba.

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_val_and_test, y_train, y_val_and_test = train_test_split(Xclean, Y, train_size=.70, shuffle=True, random_state=1)
x_val, x_test, y_val, y_test = train_test_split(x_val_and_test, y_val_and_test, test_size=.50, shuffle=True, random_state=17)

print('X,y Train:', len(x_train), len(y_train))
print('X,y Val:', len(x_val), len(y_val))
print('X,y Test', len(x_test), len(y_test))

X,y Train: 2100 2100
X,y Val: 450 450
X,y Test 450 450


#**Pregunta 4**

Usando el conjunto de entrenamiento genera un vocabulario que no sea mayor a 1500 palabras, ni menor a 1000.

¿Por qué es importante acotar un vocabulario inferior y superiormente?
¿Por qué debe usarse solamente el conjunto de entrenamiento para generar el diccionario?

Con este vocabulario que obtienes, filtra los conjuntos de entrenamiento, validación y prueba, de esta manera todos los comentarios usarán solamente palabras válidas de acuerdo a este vocabulario.

Indica el tamaño del vocabulario obtenido.

In [ ]:
from collections import Counter

In [ ]:
midiccionario = Counter()

for k in range(len(x_train)):
  midiccionario.update(x_train[k])


print('Longitud del diccionario:', len(midiccionario))
print('\n(word,frequency):')
print(midiccionario.most_common(10))

Longitud del diccionario: 3271

(word,frequency):
[('good', 201), ('great', 141), ('movie', 140), ('phone', 133), ('film', 130), ('work', 112), ('bad', 112), ('like', 101), ('time', 101), ('one', 100)]


In [ ]:
min_freq = 2    # Con 2 en general da un buen resultado. Valores más grandes empieza en este caso a generar muchos comentarios vacíos.
midicc0 = {}
midicc0 = {k:c for k,c in midiccionario.items() if c >= min_freq}

midicc = {}
midicc = {k:c for k,c in midicc0.items() if len(k)>1}

print('Nueva longitud del nuevo vocabulario:',len(midicc))
print(list(midicc.items())[0:5])

Nueva longitud del nuevo vocabulario: 1432
[('star', 18), ('dont', 59), ('fare', 2), ('much', 39), ('good', 201)]


In [ ]:
train_x = []
for ss in x_train:
  train_x.append([w for w in ss if w in midicc])

val_x = []
for ss in x_val:
  val_x.append([w for w in ss if w in midicc])

test_x = []
for ss in x_test:
  test_x.append([w for w in ss if w in midicc])

###**---Incluyen sus comentarios---**

###**Hasta aquí han hecho básicamente lo mismo que la semana pasada, salvo que aquí se les dice que necesariamente apliquen Lemmatization**

#**Pregunta - 5:**

Utilizarás los vectores embebidos FastText preentrenados por Facebook.

a. Incluye una tabla comparativa de pros y contras entre los modelos FastText, word2vec de Google y Glove de Stanford. Puedes consultar sus páginas correspondientes:

https://fasttext.cc/

https://code.google.com/archive/p/word2vec/

https://nlp.stanford.edu/projects/glove/

###**---- Incluyen Tabla con diferencias de los tres métodos embebidos----**

#**Pregunta - 6:**

Utiliza el modelo FastText de vectores embebidos pre-entrenados de dimensión 300 para generar un nuevo diccionario clave-valor, donde la “clave” será cada token o palabra de tu vocabulario y el “valor” será su vector embebido de dimensión 300.

Este diccionario deberá ser del mismo tamaño que el vocabulario previo que hayas construido previamente.

https://fasttext.cc/docs/en/crawl-vectors.html

##**Hasta aquí tenemos casi 2GB de RAM...**

In [ ]:
!pip install fasttext                  # tarda unos 1.5 mins

In [ ]:
import fasttext
import fasttext.util

In [ ]:
# English ... downloading cc.en.300.bin.gz  +++++++++++++++        ENGLISH    ++++++++++++          ... tarda unos 30 mins

fasttext.util.download_model('en', if_exists='ignore')

'cc.en.300.bin'

In [ ]:
ft = fasttext.load_model('cc.en.300.bin')   # tarda poco menos de 1 min  ... aquí es donde sube la RAM, al cargarlos en memoria.
                                            # La RAM está en 2GB y al ejecutarlo ¡¡sube como a 9GB!!

In [ ]:
print('Tamaño del vocabulario de FastText:', len(ft.words) )
print('Tamaño de los vectores embebidos FastText:', ft.get_dimension())

Tamaño del vocabulario de FastText: 2000000
Tamaño de los vectores embebidos FastText: 300


In [ ]:
from numpy import savetxt

In [ ]:
# Guardemos nuestro vocabulario-diccionario {(key:value)} --> {(word:300_embedding_vector)}

dicc_embed = {}

for w,_ in midicc.items():
  dicc_embed.update({w:ft.get_word_vector(w)})   # al parecer por default encuentra el vector más cercano...

# Guardamos el archivo:
import pickle

with open("myfileFasttextEmbed.pkl", "wb") as tf:
    pickle.dump(dicc_embed,tf)



In [ ]:
list(dicc_embed.items())[0:2]   # veamos un par de ellos...

[('star',
  array([-2.86828041e-01,  1.15123533e-01,  5.38213179e-02,  6.26803637e-02,
          2.47128960e-02, -2.57342309e-02,  1.47269234e-01, -1.16852485e-01,
          1.43789779e-02,  2.02018499e-01, -6.00243136e-02, -6.75183982e-02,
         -4.77372715e-03,  4.89413664e-02,  1.16688116e-02, -3.23957461e-03,
         -1.23359617e-02, -8.51226449e-02, -4.92033921e-02,  6.35383427e-02,
          8.86741839e-03,  1.40952796e-01,  1.09847665e-01,  1.25429276e-02,
         -4.25020084e-02, -8.08417723e-02,  2.70867231e-03,  3.24510969e-02,
          1.41084492e-02,  1.06206566e-01,  1.08138800e-01, -5.41383438e-02,
          5.69636673e-02,  5.37327491e-02, -7.06092343e-02, -9.78406295e-02,
         -1.20484188e-01, -4.16465327e-02, -8.21356028e-02,  4.71536033e-02,
          9.90687776e-03,  4.89758067e-02, -1.33870468e-01, -1.54928997e-01,
         -1.81018680e-01,  1.78726465e-02,  7.79893203e-03, -8.59202892e-02,
         -2.29937509e-02, -1.81513757e-01, -1.04256801e-01,  1.193

In [ ]:
# Borramos el ft de FastText ... observa la cantidad de RAM que se va a liberar y vuelve a bajar nuevamente :)
# En dado caso no lo borres hasta que estás seguro que ya hiciste todo lo necesario hata esta etapa.
# Baja de poco más de 9GB a poco más de 2GB ...

del ft

#**Pregunta - 7:**

Una manera de utilizar los vectores embebidos con modelos de aprendizaje automático en documentos de texto, es asignar a cada comentario filtrado el vector embebido de dimensión 300 que resulta de promediar todos sus tokens.

Así, en este ejercicio deberás generar los arreglos correspondientes para los conjuntos de entrenamiento, validación y prueba.

Los llamaremos trainEmb, valEmb y testEmb, respectivamente.

¿Cuáles son sus dimensiones?

¿Se podrían usar para su representación matrices dispersas (sparse-matrices) como en el caso de la matriz Tf-idf?

Responde a dichas preguntas.

In [ ]:
# Tomamos cualquier palabra del vocabulario FastText para extraer la información
 # (aunque en este caso ya sabemos que son de dimensión 300):

cols = dicc_embed['ability']
cols = cols.shape[0]
#print(cols)

rows = len(train_x)
#print(rows)

# Cada renglón de esta nueva matriz es un vector embebido de 300 coordenadas que contiene la información del comentario de dicho renglón:
train_xEmb = np.zeros((rows,cols))
train_xEmb.shape

(2100, 300)

In [ ]:
# Generamos nuestros datos de entrenamiento que serán la entrada a los modelos de machine learning:

for j,ss in enumerate(train_x):

  m = np.zeros_like(dicc_embed['absolutely'])  # inicializamos con cualquier vector embebido...
  k=0

  for w in ss:
    m += dicc_embed[w]     # acumulamos/sumamos todos los vectores embebidos que están en cada comentario...
    k += 1

  m = m / (k + 1e-15)   # promediamos y asignamos dicho vector a nuestra nueva matriz de datos de entrada train_xEmb
  train_xEmb[j,:] = m

In [ ]:
# Hacemos lo mismo para los vectores de validación...

rows = len(val_x)
val_xEmb = np.zeros((rows,cols))
val_xEmb.shape

for j,ss in enumerate(val_x):

  m = np.zeros_like(dicc_embed['absolutely'])
  k=0

  for w in ss:
    m += dicc_embed[w]
    k += 1

  m = m / (k + 1e-15)
  val_xEmb[j,:] = m






# y para lo vectores del conjunto Test:

rows = len(test_x)
test_xEmb = np.zeros((rows,cols))
test_xEmb.shape

for j,ss in enumerate(test_x):

  m = np.zeros_like(dicc_embed['absolutely'])
  k=0

  for w in ss:
    m += dicc_embed[w]
    k += 1

  m = m / (k + 1e-15)
  test_xEmb[j,:] = m

#**Pregunta - 8:**

Utiliza los modelos de regresión logística y bosque aleatorio (random forest) y encuentra sus desempeños (accuracy).

Compara los resultados con los de la semana anterior.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# REGRESIÓN LOGÍSTICA:

modeloLRemb = LogisticRegression(max_iter=1000,     # con estos valores se mantiene dentro del 2% que usualmente les digo...
                                 C=0.3,
                                 penalty='l2',
                                 solver='saga'
                                 )
modeloLRemb.fit(train_xEmb, y_train)

print('\nLR: Train-accuracy: %.2f%%' % (100*modeloLRemb.score(train_xEmb, y_train)))
print('LR: Val-accuracy: %2.f%%' % (100*modeloLRemb.score(val_xEmb, y_val)))


LR: Train-accuracy: 79.95%
LR: Val-accuracy: 78%


In [ ]:
modeloRFemb = RandomForestClassifier(max_depth=17,
                                  min_samples_split=14,
                                  ccp_alpha=0.01,
                                  min_samples_leaf=1,
                                  n_estimators=100,
                                  random_state=1)
modeloRFemb.fit(train_xEmb, y_train)

print('\nRF: Train-accuracy: %.2f%%' % (100*modeloRFemb.score(train_xEmb, y_train)))
print('RF: Val-accuracy: %.2f%%' % (100*modeloRFemb.score(val_xEmb, y_val)))


RF: Train-accuracy: 78.57%
RF: Val-accuracy: 76.44%


In [ ]:
78.57-76.44    # aproximadamente dentro del 2% de diferencia....

2.1299999999999955

#**Pregunta - 9:**

Obtener la mejor matriz de confusión de estos dos modelos e interpreta sus resultados...

(**NOTA para el profesor:** En realidad es suficiente con la matriz del mejor modelo):

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
pred = modeloLRemb.predict(test_xEmb)    # usando la regresión logística en este caso...
print('\nMatriz de confusión del LR:')
print(confusion_matrix(y_test, pred, labels=[0,1]))

print('\nMatriz de confusión de LR en proporciones:')
print(confusion_matrix(y_test, pred, labels=[0,1]) / pred.shape[0])


Matriz de confusión del LR:
[[166  50]
 [ 46 188]]

Matriz de confusión de LR en proporciones:
[[0.36888889 0.11111111]
 [0.10222222 0.41777778]]
